<a href="https://colab.research.google.com/github/assafsinvani/idc_nlp/blob/master/NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [34]:
import json
from pathlib import Path
from glob import glob

def preprocess():
  vocabulary = set()
  pathlist = glob('/content/drive/My Drive/idc/nlp/ex1/*.json')
  for path in pathlist: 
    with open(path) as json_file:
      data = json.load(json_file)
      for text in data['tweet_text'].values():
        vocabulary.update(list(text))
  # vocabulary.discard(' ')
  # vocabulary.discard('\t')
  # vocabulary.discard('\r')
  # vocabulary.discard('\n')

  return sorted(list(vocabulary))

vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)
  

1857
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', 'ʖ', 'ʰ', 'ʳ', 'ʷ', 'ʸ', 'ˍ

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [62]:
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  model = {}
  with open(data_file_path) as json_file:
    data_file = json.load(json_file)
    for data in data_file['tweet_text'].values():
      data = (" " * n) + data
      for i in range(len(data)-n):
        word, char = data[i:i+n], data[i+n]
        if not word in model:
          model[word] = {}
        model[word][char] = model[word].get(char, 0) + 1
      def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    model.update({hist:normalize(chars) for hist, chars in model.items()})

  return model

model = lm(2, vocabulary, '/content/drive/My Drive/idc/nlp/ex1/en.csv.json', True)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})

{'  ': [('R', 0.4323219105258853), (' ', 0.044328126277909545), ('N', 0.010550421198985851), ('@', 0.12284288868896703), ('G', 0.00785147624110575), ('I', 0.03500449824159647), ('C', 0.0131675799460211), ('W', 0.016684387012349717), ('S', 0.020201194078678333), ('h', 0.02420871841007606), ('w', 0.003107876012104359), ('i', 0.006215752024208718), ('L', 0.011122924674899812), ('⚡', 0.0013085793735176249), ('A', 0.015539380060521796), ('B', 0.008587552138709414), ('t', 0.0050707450723807964), ('E', 0.0050707450723807964), ('👈', 0.0002453586325345547), ('r', 0.0011450069518279219), ('s', 0.0024535863253455467), ('*', 0.0016357242168970312), ('#', 0.031814836018647256), ('T', 0.029851966958370817), ('M', 0.013576511000245359), ('Z', 0.0005725034759139609), ('Y', 0.00989613151222704), ('b', 0.0017992966385867342), ('"', 0.005888607180829312), ('n', 0.0009814345301382187), ('K', 0.0015539380060521795), ('F', 0.008505765927864561), ('O', 0.007606117608571195), ('【', 0.0001635724216897031), ('H

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [0]:
def eval(n, model, data_file):
  # n - the n-gram that you used to build your model (must be the same number)
  # model - the dictionary (model) to use for calculating perplexity
  # data_file - the tweets file that you wish to claculate a perplexity score for

  # TODO

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [0]:
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not

  #TODO

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [0]:
# TODO

# **Good luck!**